In [1]:
from sportsref.baseball import Franchise
from sportsref.baseball.util import get_frans

im calling bref now


In [9]:
import pandas as pd

In [21]:
def win_over_500(col):
    return int(col.split("-")[0]) - int(col.split("-")[1])

In [53]:
def fran_schedule(fran, start_year):
    df_lst = []
    fran = Franchise(fran)
    for year in range(start_year, 2020):
        df = fran.season(year).schedule_results()
        df["team"] = fran
        df["year"] = year
        df_lst.append(df)
    return pd.concat(df_lst)

In [57]:
## this will take a while
big_df = pd.concat([ fran_schedule(fran, 2010) for fran in get_frans().abbr])

In [78]:
big_df["W"] = (big_df["W/L"].str.slice(0) == "W") * 1
big_df["L"] = (big_df["W/L"].str.slice(0) == "L") * 1

In [79]:
big_df["idx"] = big_df.year.map(str) + big_df["Gm#"].astype(str).str.zfill(3)

In [129]:
df = big_df.groupby(["team", "year", "Gm#"])[["W", "L"]].sum().groupby(level=[0])[["W","L"]].cumsum()

In [132]:
out_df = df.reset_index()

In [147]:
out_df["idx"] = out_df.year.map(str) + "-" + out_df["Gm#"].astype(str).str.zfill(3)

In [148]:
out_df["above_500"] = out_df.W - out_df.L

In [150]:
out_df = out_df.fillna(method="ffill")

In [154]:
out_df.pivot(index="team", columns="idx", values="above_500").to_csv("2010s-wins_above_500.csv")

In [162]:
out=pd.concat([out_df.pivot(index="team", columns="idx", values="above_500").max(axis=1),out_df.pivot(index="team", columns="idx", values="above_500").min(axis=1)], axis=1)

In [164]:
out.columns = ["apex", "nadir"]
print(out.assign(diff=lambda x: x.apex - x.nadir).to_markdown())

| team   |   apex |   nadir |   diff |
|:-------|-------:|--------:|-------:|
| ANA    |     57 |     -11 |     68 |
| ARI    |      9 |     -64 |     73 |
| ATL    |     88 |      -7 |     95 |
| BAL    |     20 |    -120 |    140 |
| BOS    |    131 |      -6 |    137 |
| CHC    |     16 |    -119 |    135 |
| CHW    |     34 |    -119 |    153 |
| CIN    |     70 |     -54 |    124 |
| CLE    |     84 |     -65 |    149 |
| COL    |     15 |    -121 |    136 |
| DET    |    101 |     -39 |    140 |
| FLA    |      3 |    -200 |    203 |
| HOU    |     -1 |    -180 |    179 |
| KCR    |     -1 |    -112 |    111 |
| LAD    |    205 |     -19 |    224 |
| MIL    |     33 |     -29 |     62 |
| MIN    |     30 |    -139 |    169 |
| NYM    |     17 |     -45 |     62 |
| NYY    |    226 |      -1 |    227 |
| OAK    |     62 |     -21 |     83 |
| PHI    |     75 |     -59 |    134 |
| PIT    |     10 |     -73 |     83 |
| SDP    |     22 |    -141 |    163 |
| SEA    |      4 |     -